imports

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils import create_dataloaders
from utils import plot_tsne
import AutoDecoder as AD

In [22]:
import importlib

In [23]:
import utils as utils
importlib.reload(AD)
importlib.reload(utils)


<module 'utils' from '/home/nativ/Deep_learning_technion/Project - Autodecoder/utils.py'>

In [24]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)
# Define model parameters
latent_dim = 128  # Dimension of latent space
output_shape = (1, 28, 28)  # Output shape for Fashion MNIST images

# Initialize model
auto_decoder = AD.AutoDecoder(latent_dim, output_shape).to(device)


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
# Create data loaders
train_ds, train_dl, test_ds, test_dl = create_dataloaders(data_path="dataset", batch_size=32)

In [6]:
# Initialize latent vectors
train_latents = torch.randn(len(train_ds), latent_dim, requires_grad=True, device=device)
test_latents = torch.randn(len(test_ds), latent_dim, requires_grad=True, device=device)


In [ ]:
plot_tsne(train_ds, train_latents, "part1_train_tsne_before.png", plot_title="t-SNE Plot")

In [ ]:

plot_tsne(test_ds, test_latents, "part1_test_tsne_before.png", plot_title="t-SNE Plot")

In [9]:
# # Split train_ds into train and validation sets
# from torch.utils.data import random_split
# val_size = len(train_ds) // 5  # 20% for validation
# train_size = len(train_ds) - val_size
# train_subset, val_subset = random_split(train_ds, [train_size, val_size])

# train_dl = torch.utils.data.DataLoader(train_subset, batch_size=16, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_subset, batch_size=16, shuffle=False)
# # Initialize latent vectors
# train_latents = torch.randn(len(train_subset), latent_dim, requires_grad=True, device=device)
# val_latents = torch.randn(len(val_subset), latent_dim, requires_grad=False, device=device)
# test_latents = torch.randn(len(test_ds), latent_dim, requires_grad=False, device=device)

In [10]:
# Initialize optimizer
optimizer = optim.Adam(list(auto_decoder.parameters()) + [train_latents], lr=1e-3)

In [ ]:
auto_decoder.load_state_dict(torch.load('auto_decoder_parameters.pth'))
# train_losses = AD.train_auto_decoder(auto_decoder, train_dl, optimizer, train_latents, device, epochs=100)
# # save auto_decoder parameters
# torch.save(auto_decoder.state_dict(), 'auto_decoder_parameters.pth')

In [ ]:
#AD.plot_learning_curve(train_losses)

In [ ]:
AD.visualize_reconstructions(auto_decoder, train_dl, train_latents, device, num_samples=5)

In [ ]:
plot_tsne(train_ds, train_latents, "part1_train_tsne.png", plot_title="t-SNE Plot")

In [15]:
test_latents_optimizer = optim.Adam([test_latents], lr=1e-2)

In [ ]:
from evaluate import evaluate_model

evaluate_model(auto_decoder, test_dl, test_latents_optimizer, test_latents, epochs=100, device=device)

In [ ]:
plot_tsne(test_ds, test_latents, "part1_test_tsne.png", plot_title="t-SNE Plot")

In [ ]:
AD.visualize_reconstructions(auto_decoder, test_dl, test_latents, device,num_samples=5)

In [19]:
random_latents = torch.randn(5, latent_dim, requires_grad=False, device=device)

In [ ]:
AD.visualize_from_latents(auto_decoder, random_latents, device,num_samples=5)
